In [2]:
import jieba
import pandas as pd
import random
cate_dic = {'technology':1, 'car':2, 'entertainment':3, 'military':4, 'sports':5}

df_technology = pd.read_csv("../data/technology_news.csv", encoding='utf-8')
df_technology = df_technology.dropna()

df_car = pd.read_csv("../data/car_news.csv", encoding='utf-8')
df_car = df_car.dropna()

df_entertainment = pd.read_csv("../data/entertainment_news.csv", encoding='utf-8')
df_entertainment = df_entertainment.dropna()

df_military = pd.read_csv("../data/military_news.csv", encoding='utf-8')
df_military = df_military.dropna()

df_sports = pd.read_csv("../data/sports_news.csv", encoding='utf-8')
df_sports = df_sports.dropna()

technology = df_technology.content.values.tolist()[1000:21000]
car = df_car.content.values.tolist()[1000:21000]
entertainment = df_entertainment.content.values.tolist()[:20000]
military = df_military.content.values.tolist()[:20000]
sports = df_sports.content.values.tolist()[:20000]

In [3]:
stopwords=pd.read_csv("../data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values
def preprocess_text(content_lines, sentences, category):
    for line in content_lines:
        try:
            segs=jieba.lcut(line)
            segs = filter(lambda x:len(x)>1, segs)
            segs = filter(lambda x:x not in stopwords, segs)
            sentences.append("__label__"+str(category)+" , "+" ".join(segs))
        except :
            print(line)
            continue 

In [4]:
#生成训练数据
sentences = []

preprocess_text(technology, sentences, cate_dic['technology'])
preprocess_text(car, sentences, cate_dic['car'])
preprocess_text(entertainment, sentences, cate_dic['entertainment'])
preprocess_text(military, sentences, cate_dic['military'])
preprocess_text(sports, sentences, cate_dic['sports'])

random.shuffle(sentences)

Building prefix dict from the default dictionary ...
Loading model from cache C:\WINDOWS\TEMP\jieba.cache
Loading model cost 0.844 seconds.
Prefix dict has been built successfully.


In [5]:
print("writing data to fasttext format...")
out = open('train_data.txt', 'w')
for sentence in sentences:
    out.write(sentence+"\n")
print("done!")

writing data to fasttext format...
done!


In [7]:
import fasttext
#classifier = fasttext.train_supervised('train_data.txt', 'classifier.model', label_prefix='__label__')
module = fasttext.train_supervised("train_data.txt",lr=0.1,
    dim=200,
    epoch=50,
    neg=5,
    wordNgrams=2,
    label="__label__")
module.save_model("model_file.bin")

In [9]:
module = fasttext.load_model('model_file.bin')
label_to_cate = {'__label__1':'technology', '__label__2':'car', '__label__3':'entertainment', '__label__4':'military', '__label__5':'sports'}
result= module.test('train_data.txt')
# #print(model.test_label('train_data.txt'))
print(result)

(87574, 0.9922808139402106, 0.9922808139402106)
